In [1]:
import keras
import keras.backend as K
from keras.layers import (Input, Dense, Flatten, Dropout,Conv2D)
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.callbacks import EarlyStopping
import h5py
import numpy as np 
import numpy
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
data = h5py.File('../input/jets-dataset/PYTHIA_JETS.hdf5', 'r')
JETS = numpy.array(data["image"][:])

data2 = h5py.File('../input/jets-dataset/PYTHIA_JETS.hdf5', 'r')
JETS_label = numpy.array(data2["signal"][:])

JETS, JETS_label = shuffle(JETS, JETS_label)

JETS_label = JETS_label[0:860000].reshape(860000,1)
JETS = JETS[0:860000]

JETS[JETS < 1e-3] = 0

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(JETS, JETS_label, train_size=0.8)

X_train = X_train.astype('float32')/100.
X_test = X_test.astype('float32')/100.

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [4]:
image = Input(shape=(25, 25, 1))

x = Conv2D(32, kernel_size = (11, 11), padding='same', strides = 2)(image)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)

x = Conv2D(64, kernel_size = (5, 5), padding='same', strides = 2)(x)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)

x = Conv2D(64, kernel_size = (5, 5), padding='same', strides = 2)(x)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)

x = Flatten()(x)

x = Dense(128)(x)
x = LeakyReLU()(x)

x = Dense(64)(x)
x = LeakyReLU()(x)

x = Dense(1, activation='sigmoid')(x)

CNN = Model(image, x)
CNN.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25, 25, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 13, 13, 32)        3904      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 13, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          51264     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)         

In [5]:
CNN.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

In [6]:
CNN_hist = CNN.fit(X_train, Y_train, batch_size=100, epochs=50, verbose=1, validation_data=(X_test, Y_test), callbacks = [EarlyStopping(patience = 10)])

Epoch 1/50
6880/6880 [==============================] - 28s 4ms/step - loss: 0.4551 - accuracy: 0.7912 - val_loss: 0.4351 - val_accuracy: 0.8040
Epoch 2/50
6880/6880 [==============================] - 29s 4ms/step - loss: 0.4258 - accuracy: 0.8084 - val_loss: 0.4089 - val_accuracy: 0.8169
Epoch 3/50
6880/6880 [==============================] - 28s 4ms/step - loss: 0.4174 - accuracy: 0.8132 - val_loss: 0.4084 - val_accuracy: 0.8193
Epoch 4/50
6880/6880 [==============================] - 29s 4ms/step - loss: 0.4132 - accuracy: 0.8156 - val_loss: 0.4031 - val_accuracy: 0.8214
Epoch 5/50
6880/6880 [==============================] - 28s 4ms/step - loss: 0.4104 - accuracy: 0.8170 - val_loss: 0.4003 - val_accuracy: 0.8227
Epoch 6/50
6880/6880 [==============================] - 28s 4ms/step - loss: 0.4087 - accuracy: 0.8182 - val_loss: 0.4018 - val_accuracy: 0.8233
Epoch 7/50
6880/6880 [==============================] - 29s 4ms/step - loss: 0.4071 - accuracy: 0.8191 - val_loss: 0.3977 - val_ac

In [7]:
CNN.save('11X11_CNN_LR.h5')